In [ ]:
import requests

url = "https://zerogpt.p.rapidapi.com/api/v1/detectText"

payload = { "input_text": "C++ is a high-level, general-purpose programming language that was developed by Bjarne Stroustrup in 1983 as an extension of the C programming language. It is an object-oriented language that allows developers to write efficient and portable code that can run on a wide range of platforms, from embedded systems to supercomputers. In this article, we will discuss some of the key features and benefits of C++, as well as its various applications and use cases." }
headers = {
	"content-type": "application/json",
	"your_key": "your_key",
	"X-RapidAPI-Key": "your_api",
	"X-RapidAPI-Host": "zerogpt.p.rapidapi.com"
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())

{'success': True, 'data': {'is_human_written': 0, 'is_gpt_generated': 100, 'feedback_message': '', 'gpt_generated_sentences': ['C++ is a high-level, general-purpose programming language that was developed by Bjarne Stroustrup in 1983 as an extension of the C programming language.', 'It is an object-oriented language that allows developers to write efficient and portable code that can run on a wide range of platforms, from embedded systems to supercomputers.', 'In this article, we will discuss some of the key features and benefits of C++, as well as its various applications and use cases.'], 'words_count': 75}, 'code': 200, 'message': 'Detection complete'}


In [ ]:
response_json = response.json()
print(response_json['data']['is_gpt_generated'], response_json['data']['is_human_written'])

100 0


In [ ]:
import pandas as pd

data_path = 'test_data.txt'

with open(data_path, 'r', encoding='utf-8') as file:
    raw_data = file.read()

entries = raw_data.split('\n\n')

df = pd.DataFrame(entries, columns=['Entry'])

df[['Question', 'Answer']] = df['Entry'].str.split('\nAnswer: ', expand=True)
df['Label'] = ['human' if i % 2 == 0 else 'machine' for i in range(len(df))]

df.drop(columns=['Entry'], inplace=True)

df.head()

,Question,Answer,Label
0,Question: What are the treatments for Myelodys...,Because myelodysplastic /myeloproliferative ne...,human
1,Question: What are the treatments for Myelodys...,Treatments for. Myelodysplastic/ Myeloprolifer...,machine
2,Question: What are the symptoms of Adult Non-H...,Signs and symptoms of adult nonHodgkin lymphom...,human
3,Question: What are the symptoms of Adult Non-H...,Swollen lymph nodes. Fever. Night sweats. Wei...,machine
4,Question: What are the symptoms of Small Cell ...,Signs and symptoms of small cell lung cancer i...,human


In [ ]:
from tqdm import tqdm

def zerogpt_response(text_to_check):
    payload = {"input_text": text_to_check}
    headers = {
        "content-type": "application/json",
        "ATBBSsz93c01QNyT9YmehEvtR0QNcTeB2D42E40": "ATBBSsz93c01QNyT9YmehEvtR0QNcTeB2D42E40",
        "X-RapidAPI-Key": "eda65dbb41msh43ed1504a2737e4p10fd80jsn8a781f9949cf",
        "X-RapidAPI-Host": "zerogpt.p.rapidapi.com"
    }
    response = requests.post(url, json=payload, headers=headers)
    return response



responses = []
for prompt in tqdm(df['Answer']):
    try:
        response = zerogpt_response(prompt)
        responses.append(response.json()['data']['is_gpt_generated'])
    except:
        responses.append(2)

df['Response'] = responses
df.head()

100%|██████████| 147/147 [10:54<00:00,  4.45s/it]


,Question,Answer,Label,Response
0,Question: What are the treatments for Myelodys...,Because myelodysplastic /myeloproliferative ne...,human,94.29
1,Question: What are the treatments for Myelodys...,Treatments for. Myelodysplastic/ Myeloprolifer...,machine,0.00
2,Question: What are the symptoms of Adult Non-H...,Signs and symptoms of adult nonHodgkin lymphom...,human,82.56
3,Question: What are the symptoms of Adult Non-H...,Swollen lymph nodes. Fever. Night sweats. Wei...,machine,29.70
4,Question: What are the symptoms of Small Cell ...,Signs and symptoms of small cell lung cancer i...,human,73.10


In [ ]:
df['Response'] = df['Response'].replace(2, 50)
df.head()

,Question,Answer,Label,Response
0,Question: What are the treatments for Myelodys...,Because myelodysplastic /myeloproliferative ne...,human,94.29
1,Question: What are the treatments for Myelodys...,Treatments for. Myelodysplastic/ Myeloprolifer...,machine,0.00
2,Question: What are the symptoms of Adult Non-H...,Signs and symptoms of adult nonHodgkin lymphom...,human,82.56
3,Question: What are the symptoms of Adult Non-H...,Swollen lymph nodes. Fever. Night sweats. Wei...,machine,29.70
4,Question: What are the symptoms of Small Cell ...,Signs and symptoms of small cell lung cancer i...,human,73.10


In [ ]:
import numpy as np

df['Label_numeric'] = df['Label'].replace({'human': 100, 'machine': 0})

mse = np.mean((df['Label_numeric']/100 - df['Response']/100) ** 2)
print('Mean Squared Error:', mse)
df.to_csv('zerogpt_dataframe.csv', index=False)

Mean Squared Error: 0.32682581707483
